In [3]:
from transformers.models.llama.modeling_llama import LlamaDecoderLayer
import sys
import os
sys.path.append(os.path.abspath('../'))
import torch
from ft_datasets.alpaca_dataset.alpaca_dataset import get_alpaca_dataset
from ft_datasets.gsm8k.gsm8k_dataset import get_gsm8k_dataset
from ft_datasets.boolq_dataset.boolq_dataset import get_boolq_dataset
from ft_datasets.dolly_dataset.dolly_dataset import get_dolly_dataset
from ft_datasets.openbookqa.openbookqa_dataset import get_openbookqa_dataset
from configs.datasets import alpaca_dataset as alpaca_dataset_config
from configs.datasets import dolly_dataset as dolly_dataset_config
from configs.datasets import gsm8k_dataset as gsm8k_dataset_config
from tqdm import tqdm
from configs.datasets import boolq_dataset as boolq_dataset_config
from configs.datasets import openbookqa_dataset as openbookqa_dataset_config
from configs.datasets import openbookqa_dataset as openbookqa_dataset_config
import transformers
from transformers import AutoModel,AutoTokenizer,AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")




#model.to(torch.bfloat16)
tokenizer=AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
# dolly_dataset_config.data_path = "../ft_datasets/dolly_dataset/dolly-15k-test.jsonl"
# dataset=get_dolly_dataset(dolly_dataset_config, tokenizer, partition="whole",pad=False)

openbookqa_dataset_config.data_path = "../ft_datasets/openbookqa/openbookqa_test.jsonl"
dataset=get_openbookqa_dataset(openbookqa_dataset_config, tokenizer, partition="test",pad=False)

print(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

dataset_config.data_path ../ft_datasets/openbookqa/openbookqa_test.jsonl
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_att

In [4]:
def get_input_text(idx):
    chat=dataset[idx]
    tokens = torch.tensor(chat["input_ids"]).unsqueeze(0).long()
    labels = torch.tensor(chat["labels"]).unsqueeze(0).long()
    mask = labels[0] < 0
    input_token= tokens[0][mask]
    instruction = tokenizer.decode(input_token, skip_special_tokens=False)

    #print("Instruction: ", instruction)
    return instruction

In [5]:
import warnings
warnings.filterwarnings("ignore")

def get_last_token_hidden_states(module, input, output):
    last_token_states.append(output[0][:, -1, :].detach().cpu())

# Load model and tokenizer

# Register hook
hidden_states = []
model.to("cuda:0")
last_token_states = []

# Register hooks on each transformer layer to extract hidden states
for layer in model.model.layers:
    layer.register_forward_hook(get_last_token_hidden_states)

# Prepare input and convert to suitable format


for idx in tqdm(range(len(dataset))):
    text=get_input_text(idx)
    inputs=tokenizer(
            text, return_tensors="pt", padding=False, truncation=True
        ).to("cuda:0")
    # Perform a forward pass to get outputs and hidden states
    output = model(**inputs)
    hidden_states.append(last_token_states)
    last_token_states = []


100%|██████████| 300/300 [00:27<00:00, 11.06it/s]


In [6]:
def get_input_text(idx):
    chat=dataset[idx]
    tokens = torch.tensor(chat["input_ids"]).unsqueeze(0).long()
    labels = torch.tensor(chat["labels"]).unsqueeze(0).long()
    mask = labels[0] < 0
    input_token= tokens[0][mask]
    instruction = tokenizer.decode(input_token, skip_special_tokens=False)

    #print("Instruction: ", instruction)
    return instruction

In [7]:
print(len(hidden_states))
print(len(hidden_states[0]))
print(hidden_states[0][0].shape)

tensor_hidden_states = torch.stack([torch.stack(batch, dim=0) for batch in hidden_states], dim=0)

# Reshape to (32, 300, 4096)
reshaped_hidden_states = tensor_hidden_states.permute(1, 0, 2, 3).squeeze(2)

# `reshaped_hidden_states` is now a list of 32 tensors, each of shape (300, 4096)
result = [reshaped_hidden_states[i] for i in range(reshaped_hidden_states.size(0))]

300
32
torch.Size([1, 4096])


In [8]:
print(len(result))

print(result[0].shape)
#store the hidden states
torch.save(result, "openbookqa_hidden_states.pt")

32
torch.Size([300, 4096])


In [9]:
activations = torch.load("./stored_activation/openbookqa_llama7b.pt") 

In [10]:
get_input_text(1)

'<s> [INST] <<SYS>>\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n<</SYS>>\n\n### Instruction:\n There is most likely going to be fog around:A: a marsh\nB: a tundra\nC: the plains\nD: a desert \n \n### Response: [/INST]'

In [16]:
def get_manipulated_feature(source_feature, layer_idx, eigenvector_idx, data_idx, device='cuda:0'):

    if eigenvector_idx is None:
        return source_feature
    
    X=activations[layer_idx]
    if X.dtype == torch.bfloat16:
        X = X.to(dtype=torch.float32)
    X = X.to(device)

    X_mean = X.mean(dim=0, keepdim=True)  # [1, D]
    X_centered = X - X_mean              # [N, D]

    U, S, Vt = torch.linalg.svd(X_centered, full_matrices=False)
    V = Vt.t() 
    z_i = U[data_idx, :] * S
    z_i_clean = z_i.clone()
    for pc in eigenvector_idx:
        z_i_clean[pc] = 0.0


    
    subtracted_part_centered = (z_i.unsqueeze(0)-z_i_clean.unsqueeze(0)) @ Vt  # [1, D

    
    x_clean = source_feature - 1*subtracted_part_centered
    return x_clean

In [17]:
import numpy as np
layer_num=12

layer_name=f"model.layers.{layer_num}"
model.to("cuda:0")


data_index=101
input_text = get_input_text(data_index)

layer_remove=np.arange(0,3).tolist()
# layer_remove=[]
print(layer_remove)
def change_activations(module, input, output):
    modified_output = output[0].clone()
    
    modified_output[:,  - 1, :] = get_manipulated_feature(modified_output[:,  - 1, :], layer_num, layer_remove,data_idx=data_index)
    print("change shape: ",  output[0][:,  - 1, :]-modified_output[:,  - 1, :])

    return (modified_output,) + output[1:]


for name, module in model.named_modules():
    if name.endswith(layer_name):
    
        hook = module.register_forward_hook(change_activations)

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda:0")
with torch.no_grad():
    outputs = model.generate(input_ids, max_new_tokens=150, do_sample=False, temperature=1.0,)

hook.remove()



[0, 1, 2]
change shape:  tensor([[ 0.1066,  0.0672,  0.0440,  ...,  0.0006, -0.1297, -0.0512]],
       device='cuda:0')
change shape:  tensor([[ 0.1066,  0.0672,  0.0440,  ...,  0.0006, -0.1297, -0.0512]],
       device='cuda:0')
change shape:  tensor([[ 0.1066,  0.0672,  0.0440,  ...,  0.0006, -0.1297, -0.0512]],
       device='cuda:0')
change shape:  tensor([[ 0.1066,  0.0672,  0.0440,  ...,  0.0006, -0.1297, -0.0512]],
       device='cuda:0')
change shape:  tensor([[ 0.1066,  0.0672,  0.0440,  ...,  0.0006, -0.1297, -0.0512]],
       device='cuda:0')
change shape:  tensor([[ 0.1066,  0.0672,  0.0440,  ...,  0.0006, -0.1297, -0.0512]],
       device='cuda:0')
change shape:  tensor([[ 0.1066,  0.0672,  0.0440,  ...,  0.0006, -0.1297, -0.0512]],
       device='cuda:0')
change shape:  tensor([[ 0.1066,  0.0672,  0.0440,  ...,  0.0006, -0.1297, -0.0512]],
       device='cuda:0')
change shape:  tensor([[ 0.1066,  0.0672,  0.0440,  ...,  0.0006, -0.1297, -0.0512]],
       device='cuda:0')


In [18]:
# Decode generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
print(generated_text)

<s><s>  [INST] <<SYS>>
Below is an instruction that describes a task. Write a response that appropriately completes the request.
<</SYS>>

### Instruction:
 Which animal is considered a predator?A: ant
B: snake
C: elephant
D: giraffe 
 
### Response: [/INST]  The answer is (B) snake.

Snakes are considered predators because they are carnivorous and feed on a variety of prey, including small mammals, birds, and other reptiles. They are known for their stealth and agility, which allows them to ambush and catch their prey with ease.

While ants are social insects that work together to gather food and protect their colonies, they are not predators in the classical sense. Elephants are herbivores and do not have a predatory role in their ecosystems, and giraffes are primarily grazers and do not have a strong predatory component


In [19]:
import numpy as np
layer_num=12

layer_name=f"model.layers.{layer_num}"
model.to("cuda:0")


data_index=101
input_text = get_input_text(data_index)

layer_remove=np.arange(0,1).tolist()

print(layer_remove)
def change_activations(module, input, output):
    
    modified_output = output[0].clone()
    
    modified_output[:,  - 1, :] = get_manipulated_feature(modified_output[:,  - 1, :], layer_num, layer_remove,data_idx=data_index)
    print("change shape: ",  output[0][:,  - 1, :]-modified_output[:,  - 1, :])

    return (modified_output,) + output[1:]


for name, module in model.named_modules():
    if name.endswith(layer_name):
    
        hook = module.register_forward_hook(change_activations)

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda:0")
with torch.no_grad():
    outputs = model.generate(input_ids, max_new_tokens=150, do_sample=False, temperature=1.0,)


hook.remove()



[0]
change shape:  tensor([[-0.0542, -0.0313,  0.0219,  ...,  0.0112, -0.0645, -0.0704]],
       device='cuda:0')
change shape:  tensor([[-0.0542, -0.0313,  0.0219,  ...,  0.0112, -0.0645, -0.0704]],
       device='cuda:0')
change shape:  tensor([[-0.0542, -0.0313,  0.0219,  ...,  0.0112, -0.0645, -0.0704]],
       device='cuda:0')
change shape:  tensor([[-0.0542, -0.0313,  0.0219,  ...,  0.0112, -0.0645, -0.0704]],
       device='cuda:0')
change shape:  tensor([[-0.0542, -0.0313,  0.0219,  ...,  0.0112, -0.0645, -0.0704]],
       device='cuda:0')
change shape:  tensor([[-0.0542, -0.0313,  0.0219,  ...,  0.0112, -0.0645, -0.0704]],
       device='cuda:0')
change shape:  tensor([[-0.0542, -0.0313,  0.0219,  ...,  0.0112, -0.0645, -0.0704]],
       device='cuda:0')
change shape:  tensor([[-0.0542, -0.0313,  0.0219,  ...,  0.0112, -0.0645, -0.0704]],
       device='cuda:0')
change shape:  tensor([[-0.0542, -0.0313,  0.0219,  ...,  0.0112, -0.0645, -0.0704]],
       device='cuda:0')
change

In [20]:
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
print(generated_text)

<s><s>  [INST] <<SYS>>
Below is an instruction that describes a task. Write a response that appropriately completes the request.
<</SYS>>

### Instruction:
 Which animal is considered a predator?A: ant
B: snake
C: elephant
D: giraffe 
 
### Response: [/INST]  The answer is (B) snake.</s>


In [21]:
import numpy as np
layer_num=12

layer_name=f"model.layers.{layer_num}"
model.to("cuda:0")


data_index=101
input_text = get_input_text(data_index)

layer_remove=[]
print(layer_remove)
def change_activations(module, input, output):
    # Clone the output to avoid in-place modifications
    modified_output = output[0].clone()
    
    # Modify only the last token's activations in the output
    #modified_output[:,  - 1, :] *= 200 # Scale activations of the last token
    modified_output[:,  - 1, :] = get_manipulated_feature(modified_output[:,  - 1, :], layer_num, layer_remove,data_idx=data_index)
    print("change shape: ",  output[0][:,  - 1, :]-modified_output[:,  - 1, :])

    # Return a new tuple with the modified output and the rest of the original outputs
    return (modified_output,) + output[1:]


for name, module in model.named_modules():
    if name.endswith(layer_name):
    
        hook = module.register_forward_hook(change_activations)

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda:0")
with torch.no_grad():
    outputs = model.generate(input_ids, max_new_tokens=150, do_sample=False, temperature=1.0,)


hook.remove()



[]
change shape:  tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
change shape:  tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
change shape:  tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
change shape:  tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
change shape:  tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
change shape:  tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
change shape:  tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
change shape:  tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
change shape:  tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
change shape:  tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
change shape:  tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
change shape:  tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
change shape:  tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
change shape:  tensor([[0., 0., 0.,  ..., 0., 0.

In [22]:
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
print(generated_text)

<s><s>  [INST] <<SYS>>
Below is an instruction that describes a task. Write a response that appropriately completes the request.
<</SYS>>

### Instruction:
 Which animal is considered a predator?A: ant
B: snake
C: elephant
D: giraffe 
 
### Response: [/INST]  Sure! Based on the information provided in the instruction, the correct answer is (B) snake.</s>
